In [15]:
import numpy as np
import requests
import re

In [16]:
def llm(question, prompts=None, examples=None):
    response = requests.post(
        "http://172.19.8.33:8080/generate/",
        json={
            "text": f"{question}",
            "generate_tokens_limit": 2048 - 1311,
            "top_p": 1,
            "top_k": 0,
            "temperature": 0.2,
            "repetition_penalty": 1.5,
            "prompt": prompts,
            "examples": examples
            },
        headers={
            "Content-Type": "application/json"
            }
        )
    return response.json()

In [17]:
seq = np.random.normal(loc=0, scale=1, size=5).round(2).tolist()
seq = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1000]
question = f"Are there any outliers in the sequence {seq}?"

In [18]:
import outlier_env


def cleaning_peter(question, prompts=None, examples=None):#(idx=None, prompt=prompt, to_print=True):
    env = outlier_env.OutlierEnv()
    question = examples + " \nQuestion: " + question
    i = 1
    
    # for i in range(1, 8):
    thought_action = llm(question=question + f"\nThought {i}: ", prompts=prompts)["completion"]

    print("-------------THOUGHT ACTION BEFORE------------")
    print(thought_action)
    print("^^^^^^^^^^^^^THOUGHT ACTION BEFORE^^^^^^^^^^^^^")

    thought_index_list = [idx.start() for idx in re.finditer(f"Thought {i}", thought_action)]
    obs_index_list = [idx.start() for idx in re.finditer(f"Observation {i}", thought_action)]

    thought_action = thought_action[thought_index_list[1]:obs_index_list[1]]

    print("-------------THOUGHT ACTION AFTER-------------")
    print(thought_action)
    print("^^^^^^^^^^^^^THOUGHT ACTION AFTER^^^^^^^^^^^^^")

    thought, action = thought_action.strip().split(f"\nAction {i}: ")
    print(thought, action)

    env.step(action=action)

    step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {env.obs}\n"
    question += step_str
    
    print(env.outliers)

    print("--------UPDATED QUESTION IS:----------------")
    print(question)



    #     try:
    #         thought, action = thought_action.strip().split(f"\nAction {i}: ")
    #     except:
    #         print('ohh...', thought_action)
    #         n_badcalls += 1
    #         n_calls += 1
    #         thought = thought_action.strip().split('\n')[0]
    #         action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
    #     obs, r, done, info = step(env, action[0].lower() + action[1:])
    #     obs = obs.replace('\\n', '')
    #     step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
    #     prompt += step_str
    #     if to_print:
    #         print(step_str)
    #     if done:
    #         break
    # if not done:
    #     obs, r, done, info = step(env, "finish[]")
    # if to_print:
    #     print(info, '\n')
    # info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    # return r, info

In [19]:
question

'Are there any outliers in the sequence [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1000]?'

In [20]:
output = cleaning_peter(question=question, examples=examples)

-------------THOUGHT ACTION BEFORE------------

                
Question: Are there any outliers in the sequence [1.3, -0.0, 1.2, -0.2, -1.0]?
                
Thought 1: I need to find outliers in the sequence.
                
Action 1: outlier_detection_step[[1.3, -0.0, 1.2, -0.2, -1.0]]
                
Observation 1: The outliers are {0: False, 1: False, 2: False, 3: False, 4: False}
                
Thought 2: The entries [] are the outliers in the sequence
                
Action 2: finish []
                 
Question: Are there any outliers in the sequence [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1000]?
Thought 1: 
                
Action 1: outlier_detection_step[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1000]]
                
Observation 1: The outliers are {0: False, 1: False, 2: False, 3: False, 4: False, 5: False, 6: False, 7: False, 8: False, 9: False, 1000: False}
                
Thought 2: The entries [] are the outliers in the sequence
                
Action 2: finish []
             

In [21]:
print(output)

None


___
___
___
# Generating examples

In [22]:
def outlier_detection_step(sequence: list[float]) -> dict[int: bool]:
    """Given a list of numbers, return a dictionary with the indices of the sequence and a boolean value indicating whether they are
    outliers or not"""
    mean = np.mean(sequence)
    sd = np.std(sequence)
    
    outliers = {idx: (abs(value - mean) > 3 * sd) for idx, value in enumerate(sequence)}
    only_outliers = [idx for idx, value in outliers.items() if value == True]
    
    return outliers, only_outliers

In [23]:
examples = ""
for i in range(1, 2):
    seq = np.random.normal(loc=0, scale=1, size=5).round(1).tolist()
    examples += f"""
                \nQuestion: Are there any outliers in the sequence {seq}?
                \nThought 1: I need to find outliers in the sequence.
                \nAction 1: outlier_detection_step[{seq}]
                \nObservation 1: The outliers are {outlier_detection_step(seq)[0]}
                \nThought 2: The entries {outlier_detection_step(seq)[1]} are the outliers in the sequence
                \nAction 2: finish {[seq[idx] for idx in outlier_detection_step(seq)[1]]}
                """

In [24]:
print(examples)


                
Question: Are there any outliers in the sequence [-0.2, 0.8, -0.7, -0.7, 1.2]?
                
Thought 1: I need to find outliers in the sequence.
                
Action 1: outlier_detection_step[[-0.2, 0.8, -0.7, -0.7, 1.2]]
                
Observation 1: The outliers are {0: False, 1: False, 2: False, 3: False, 4: False}
                
Thought 2: The entries [] are the outliers in the sequence
                
Action 2: finish []
                


___
___
___